In [1]:
## ALL LISTINGS

COLLEGE_LISTINGS = ['FIU.json', 'UGA.json', 'FSU.json', 'GSU.json' , 'GT.json', 'KSU.json']


Actually Cleaning the Uploaded Listings to Key only the relevant Keys

In [2]:
from cleaning_utils import clean_college_lisitings
for name in COLLEGE_LISTINGS:
    collegeName = name.split('.')[0]
    clean_college_lisitings(collegeName)


Filtered JSON saved to filtered_output.json
Filtered JSON saved to filtered_output.json
Filtered JSON saved to filtered_output.json
Filtered JSON saved to filtered_output.json
Filtered JSON saved to filtered_output.json
Filtered JSON saved to filtered_output.json


In [7]:
#name dataset_facebook-groups-scraper_2025-05-24_12-04-37-308.json

import json
from tqdm import tqdm

# Define your keyword filters
KEYWORDS = ['sublease', 'subleasing', 'roommate', 'roommates', 'apartment', 'apartments', 'rent', 'room']

def post_matches(text, keywords):
    """Return True if any keyword is found in the text (case-insensitive)."""
    text_lower = text.lower()
    return any(kw in text_lower for kw in keywords)

def filter_listings(input_json_path, output_json_path):
    # Load the data
    with open(input_json_path, 'r', encoding='utf-8') as f:
        listings = json.load(f)

    filtered = []
    # Iterate with a tqdm progress bar
    for post in tqdm(listings, desc="Filtering posts"):
        # Assume each post has a 'message' or 'text' field
        text = post.get('message') or post.get('text') or ''
        if post_matches(text, KEYWORDS):
            filtered.append(post)

    # Save the filtered results
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(filtered, f, ensure_ascii=False, indent=2)

    print(f"\nFound {len(filtered)} matching posts. Saved to {output_json_path}")




In [8]:
#INPUT_FILE = "removed_irrelevant_keys_output.json"

for input in COLLEGE_LISTINGS:
    file_dir = f"dataFiles/{input}"
    output_name = f"actualSubleases/{input.split('.')[0]}_actual_sublease_output.json"
    filter_listings(file_dir, output_name)

Filtering posts: 100%|██████████| 200/200 [00:00<00:00, 219654.57it/s]


Found 170 matching posts. Saved to actualSubleases/FIU_actual_sublease_output.json


Filtering posts: 100%|██████████| 200/200 [00:00<00:00, 61016.93it/s]



Found 99 matching posts. Saved to actualSubleases/UGA_actual_sublease_output.json


Filtering posts: 100%|██████████| 200/200 [00:00<00:00, 129513.79it/s]


...


Found 153 matching posts. Saved to actualSubleases/GSU_actual_sublease_output.json


Filtering posts: 100%|██████████| 200/200 [00:00<00:00, 113528.33it/s]



Found 129 matching posts. Saved to actualSubleases/GT_actual_sublease_output.json


Filtering posts: 100%|██████████| 200/200 [00:00<00:00, 66156.21it/s]


Found 181 matching posts. Saved to actualSubleases/KSU_actual_sublease_output.json


After we have sieved out all irrelevant listings which dont have to do with apartments, time to Upload the images to firestore then remove the irrelevant fields

In [9]:
import requests
import json
from typing import Dict, List, Any

API_BASE_URL = "http://localhost:5000"

def test_from_file_json(inputFile, outputFile):
    # Charger le fichier JSON
    with open(inputFile, 'r', encoding='utf-8') as f:
        listings = json.load(f)

    # Envoyer à l’API Flask
    response = requests.post(f'{API_BASE_URL}/upload-images', json=listings)

    # Vérifier la réponse
    if response.ok:
        result = response.json()
        print("✅ Traitement réussi :", result['message'])
        
        # Sauvegarder la nouvelle version avec les images mises à jour
        with open(outputFile, 'w', encoding='utf-8') as f:
            json.dump(result['listings'], f, indent=2, ensure_ascii=False)
        print("✅ Fichier output.json généré avec succès")
        return True
    else:
        print("❌ Erreur :", response.status_code, response.text)
        return False

In [11]:
SUB_COLLEGE = COLLEGE_LISTINGS[-3:]
for input in SUB_COLLEGE:
    file_dir = f"actualSubleases/{input.split('.')[0]}_actual_sublease_output.json"
    output_name = f"uploadFiles/{input.split('.')[0]}_uploaded.json"
    test_from_file_json(file_dir, output_name)

✅ Traitement réussi : Processed 153 listings. 153 successful, 0 failed. 803 images downloaded.
✅ Fichier output.json généré avec succès
✅ Traitement réussi : Processed 129 listings. 129 successful, 0 failed. 623 images downloaded.
✅ Fichier output.json généré avec succès
✅ Traitement réussi : Processed 181 listings. 181 successful, 0 failed. 325 images downloaded.
✅ Fichier output.json généré avec succès


In [4]:
def collect_non_matching(input_json_path, output_json_path=None):
    """Return a list of posts that do NOT contain any of the KEYWORDS.
       Optionally save them out to a JSON file if output_json_path is given."""
    with open(input_json_path, 'r', encoding='utf-8') as f:
        listings = json.load(f)

    non_matching = []
    for post in tqdm(listings, desc="Collecting non-matches"):
        text = post.get('message') or post.get('text') or ''
        if not post_matches(text, KEYWORDS):
            non_matching.append(post)

    if output_json_path:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(non_matching, f, ensure_ascii=False, indent=2)
        print(f"Saved {len(non_matching)} non-matching posts to {output_json_path}")

    return non_matching
# This is kinda a Sanity check cause the sum of its values plus the ones above have to give the total number of initial Listings
if __name__ == "__main__":
    non_matches = collect_non_matching("filtered_output.json", "new_non_matching.json")
    print(f"Total non-matching posts: {len(non_matches)}")

Saved 7 non-matching posts to new_non_matching.json
Total non-matching posts: 7


In [6]:
#This is just a Sanity check, it should always be 0 if everything was done right
if __name__ == "__main__":
    filter_listings("new_non_matching.json", "filtered_nonmatching_subleases.json")

Filtering posts: 100%|██████████| 7/7 [00:00<00:00, 69245.58it/s]


Found 0 matching posts. Saved to filtered_nonmatching_subleases.json
